In [4]:
from collections import namedtuple
import requests
from socketIO_client import SocketIO
import logging
import json
import uuid
import threading
from dateutil.parser import parse
from datetime import datetime
import time
import types
import configparser 

In [9]:
pip install socketIO-client

  Created wheel for socketIO-client: filename=socketIO_client-0.7.2-py3-none-any.whl size=23722 sha256=18db013fff6475ca66490c6d06836b5b583ae53d26486784be01e1dc488c33ae
  Stored in directory: c:\users\ray\appdata\local\pip\cache\wheels\0d\b6\e4\2cf25a1a9420b747705ace9acf39eb9d8afafab56d00f59501
Successfully built socketIO-client
Note: you may need to restart the kernel to use updated packages.


In [5]:
import json
from importlib import reload
import fxcm_rest_api_token as fxcm_rest_api
# import loggin
result = reload(fxcm_rest_api)

In [6]:
con = configparser.ConfigParser()
con.read('config.ini')
token = con['FXCM']['token']

'0e80fcaad559218c5ab41bb1e24620e59f1a6914'

In [9]:
trader = fxcm_rest_api.Trader(token, 'real')
trader.debug_level = "error" # verbose logging... don't set to receive errors only
trader.login()
### Wait until connection is established - you will see "Websocket connected" message

ConnectionError: OPEN packet not returned by server

In [ ]:
c =trader.candles("EUR/USD", "m15", 45, dt_fmt="%Y/%m/%d %H:%M:%S")['candles']
print(len(c))
for candle in c:
    print(candle)

In [ ]:
help(trader.open_trade)

In [ ]:
res = trader.open_trade(trader.account_id, "USD/JPY", True, 1)
orderId = res['data']['orderId']

In [ ]:
tradeId = trader.get_tradeId(orderId)

In [ ]:
print(tradeId)

In [ ]:
def print_candles(candle_data):
    candles = candle_data['candles']
    headers = candle_data['headers']
    print("{0[10]}, {0[1]}, {0[3]}, {0[4]}, {0[2]}".format(headers))
    candles.reverse()
    for candle in candles:
        print("{0[10]}, {0[1]:0.6}, {0[3]:0.6}, {0[4]:0.6}, {0[2]:0.6}".format(candle))

In [ ]:
cd = trader.candles("EUR/USD", "m15", 45, dt_fmt="%Y/%m/%d %H:%M:%S")
print_candles(cd)

In [ ]:
trader.candles_as_dict("EUR/USD", "m1", 1, dt_fmt="%Y/%m/%d %H:%M:%S")

In [ ]:
trader.accounts

In [ ]:
trader.properties

In [ ]:
print(trader.account_id)

#### Show how to supply a different handler for subscription items

In [ ]:

def show(msg):
    '''
    Sample price handler. If on_price_update is registered for a symbol,
    it will update the symbol's values (stored in a symbol hash) with
    that price update.symbol hash.

    :return: none
    '''
    try:
        md = json.loads(msg)
        symbol = md["Symbol"]
        t = trader
        si = trader.symbol_info.get(symbol, {})
        p_up = dict(symbol_info=t.symbol_info[symbol], parent=t)
        t.symbols[symbol] = t.symbols.get(symbol, fxcm_rest_api
                                          .PriceUpdate(p_up,
                                                       symbol_info=si))
        trader.symbols[symbol].bid, trader.symbols[symbol].ask,\
            trader.symbols[symbol].high,\
            trader.symbols[symbol].low = md['Rates']
        trader.symbols[symbol].updated = md['Updated']
        print(t.symbols[symbol])
    except Exception as e:
        trader.logger.error("Can't handle price update: " + str(e))

# trader = fx

In [ ]:
trader.subscribe_symbol("EUR/USD", handler=show)

In [ ]:
result = trader.subscribe_symbol("GBP/JPY")
print(result)
print(trader.subscriptions)

In [ ]:
print(trader.symbols["GBP/JPY"])
print(trader.symbols["GBP/JPY"].bid)
print(trader.symbols["GBP/JPY"].ask)

In [ ]:
print(trader.symbols['GBP/JPY'].parent)

In [ ]:
trader.open_trade(trader.account_id, "USD/JPY", True, 1, at_market=1, time_in_force="FOK")